In [ ]:
import os
import json

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
import pandas_ta as ta
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators

In [ ]:
from trading import utils

In [ ]:
ts = TimeSeries(key='ALPHAVANTAGE_API_KEY', output_format='pandas')

In [ ]:
INPUT = os.getcwd() + '/data/historical/stocks/'

In [ ]:
with open(INPUT + 'top_stocks_2015.json', 'r') as f:
    stocks = json.load(f)

In [ ]:
symbols = pd.read_csv(INPUT + 'companiesmarketcap.com - Largest CAC 40 companies by market cap.csv')['Symbol']

In [ ]:
datas = utils.load_data(INPUT)

In [ ]:
timeseries = dict()
for symbol, value in datas.items():
    data = value[0].sort_index()
    data.columns = [col[1] for col in data.columns.str.split(' ')]
    timeseries[symbol] = data

In [ ]:
multipliers = dict()
for symbol,data in timeseries.items():
    multiplier = data['close'].iloc[-1] / data['open'].iloc[0]
    multipliers[symbol] = multiplier

In [ ]:
sma_results = dict()
for symbol,data in timeseries.items():
    exposition_cond = data.ta.sma(length=100) < data['close']
    sma_results[symbol] = data['open'].diff().where(
        exposition_cond.shift()).fillna(0).cumsum() + data['open'].iloc[0]

In [ ]:
multipliers_sma = dict()
for symbol,data in sma_results.items():
    multiplier_sma = data.iloc[-1] / data.iloc[0]
    multipliers_sma[symbol] = multiplier_sma

In [ ]:
for symbol in sma_results.keys():
    print(f'Strategy for {symbol} multiply gains per {(multipliers_sma[symbol] / multipliers[symbol]):.2f}')

In [ ]:
test = sma_results['BRK.A'].index

In [ ]:
years = (test[-1] - test[0]).days / 365.25

In [ ]:
(multipliers['BRK.A'] - 1) ** (1/years) - 1, (multipliers_sma['BRK.A'] - 1) ** (1/years) - 1

In [ ]:
for symbol in timeseries.keys():
    fig, ax = plt.subplots(figsize=(10,6))
    plt.semilogy(timeseries[symbol]['close'], label=f'{symbol} price')
    plt.semilogy(sma_results[symbol], label='SMA strat')
    plt.title(f'Daily Times Series for the {symbol} stock')
    plt.legend(loc='upper left')